In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [2]:
dataset = pd.read_excel(r'd:\Jazz\2024\Python\Number 1\Aug - Dec Dataset.xlsx')

In [63]:
#1 - What is the MoM trend of Recharge, Revenue, Data Traffic, On-net Voice Traffic, Off-net Voice Traffic, Subscribers

dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Month'] = dataset['Date'].dt.month
dataset1 = dataset[['Month','City','Recharge_Amount','Total_Jazz_Revenue','Data_Traffic','Onnet_MOUs','Offnet_MOUs','Subscribers']]
dataset1 = (dataset.groupby('Month')[['Recharge_Amount','Total_Jazz_Revenue','Data_Traffic','Onnet_MOUs','Offnet_MOUs','Subscribers']].sum()/1000000).reset_index()
dataset1['Data_Traffic'] = dataset1['Data_Traffic']*1000000*(1/1024**2)*(1/30)
dataset1['Subscribers'] = dataset1['Subscribers']*(1/30)
dataset1 = dataset1.rename(columns={'Subscribers':'Daily_Avg_Subscribers','Data_Traffic':'Data_Traffic_TBs_Daily'})
print(dataset1)

   Month  Recharge_Amount  Total_Jazz_Revenue  Data_Traffic_TBs_Daily  \
0      8     21910.762663        15668.961360            10614.241246   
1      9     21713.009593        15544.346690            10388.280578   
2     10     22548.926230        16112.473149            10938.777772   
3     11     21762.141314        15804.030395            10553.107068   
4     12     22754.350122        16348.348108            11089.322782   

     Onnet_MOUs  Offnet_MOUs  Daily_Avg_Subscribers  
0  15702.569841  1610.037669              32.257955  
1  15251.109172  1552.884196              31.019879  
2  16060.655839  1647.004623              32.506596  
3  15398.447001  1596.918541              31.088709  
4  15708.607653  1663.560965              32.791511  


In [4]:
#1.1 What is the Percentage change in each KPI MoM

PCT_Change = dataset1[['Total_Jazz_Revenue','Recharge_Amount','Data_Traffic_TBs_Daily','Onnet_MOUs','Offnet_MOUs','Daily_Avg_Subscribers']].pct_change()*100
print(PCT_Change)

       Total_Jazz_Revenue  Recharge_Amount  Data_Traffic_TBs_Daily  \
Month                                                                
8                     NaN              NaN                     NaN   
9               -0.795296        -0.902539               -2.128844   
10               3.654875         3.849842                5.299214   
11              -1.914310        -3.489235               -3.525720   
12               3.444170         4.559334                5.081117   

       Onnet_MOUs  Offnet_MOUs  Daily_Avg_Subscribers  
Month                                                  
8             NaN          NaN                    NaN  
9       -2.875075    -3.549822              -3.838049  
10       5.308117     6.061007               4.792789  
11      -4.123174    -3.041041              -4.361844  
12       2.014233     4.173189               5.477236  


In [5]:
#2 - How many Cities does the dataset comprise of 
Cities = dataset['City'].drop_duplicates().count()
print(Cities)

702


In [51]:
#3 What are the Top cities comprising 80% of Revenue and Subscribers

dataset2 = pd.DataFrame(dataset[['City','Total_Jazz_Revenue']])
dataset2 = dataset2.groupby('City')['Total_Jazz_Revenue'].sum()
dataset2 = dataset2.reset_index()
dataset2['Total_Jazz_Revenue'] = dataset2['Total_Jazz_Revenue'].sort_values()
dataset2['cumulative_sum'] = dataset2['Total_Jazz_Revenue'].cumsum()
dataset2['sum_pct'] = dataset2['cumulative_sum']/(dataset2['Total_Jazz_Revenue'].sum())
dataset2['pct_cumsum'] = dataset2['sum_pct'].cumsum()
print(dataset2[dataset2['pct_cumsum'] <= 0.8])
#One index + one column = series / one index + multiple columns = dataframe

                   City  Total_Jazz_Revenue  cumulative_sum   sum_pct  \
0                     0        9.709284e+06    9.709284e+06  0.000122   
1            Abbottabad        1.468518e+08    1.565611e+08  0.001970   
2      Abbottabad_Rural        9.233303e+07    2.488941e+08  0.003132   
3         Ahmadpur East        3.794812e+07    2.868423e+08  0.003609   
4   Ahmadpur East_Rural        2.229053e+08    5.097476e+08  0.006414   
5         Ahmedpur East        6.441883e+07    5.741664e+08  0.007224   
6         Akora Khattak        1.097959e+07    5.851460e+08  0.007362   
7   Akora Khattak_Rural        9.519139e+07    6.803374e+08  0.008560   
8                Alipur        8.441927e+07    7.647566e+08  0.009622   
9         Alipur Chatha        5.128897e+07    8.160456e+08  0.010268   
10  Alipur Chatha_Rural        7.347171e+07    8.895173e+08  0.011192   
11         Alipur_Rural        1.490958e+08    1.038613e+09  0.013068   
12            AllahAbad        6.525095e+07    1.10

In [65]:
#5 Cross Visualization of MOUs/Sub with MBs/Sub

dataset3 = dataset1[['Month','Daily_Avg_Subscribers','Data_Traffic_TBs_Daily','Onnet_MOUs','Offnet_MOUs']]
dataset3['Onnet_MOUs'] = dataset3['Onnet_MOUs']*(1/30)
dataset3['Offnet_MOUs'] = dataset3['Offnet_MOUs']*(1/30)
dataset3 = dataset3.rename(columns={'Onnet_MOUs':'Daily_Avg_Onnet_MOUs','Offnet_MOUs':'Daily_Avg_Offnet_MOUs'})
dataset3['Onnet_Per_Sub'] = dataset3['Daily_Avg_Onnet_MOUs']/dataset3['Daily_Avg_Subscribers']
dataset3['Offnet_Per_Sub'] = dataset3['Daily_Avg_Offnet_MOUs']/dataset3['Daily_Avg_Subscribers']
dataset3['Data_Per_Sub'] = (dataset3['Data_Traffic_TBs_Daily']*(1024**2))/dataset3['Daily_Avg_Subscribers']
print(dataset3.head())

   Month  Daily_Avg_Subscribers  Data_Traffic_TBs_Daily  Daily_Avg_Onnet_MOUs  \
0      8              32.257955            10614.241246            523.418995   
1      9              31.019879            10388.280578            508.370306   
2     10              32.506596            10938.777772            535.355195   
3     11              31.088709            10553.107068            513.281567   
4     12              32.791511            11089.322782            523.620255   

   Daily_Avg_Offnet_MOUs  Onnet_Per_Sub  Offnet_Per_Sub  Data_Per_Sub  
0              53.667922      16.226044        1.663711  3.450262e+08  
1              51.762807      16.388533        1.668698  3.511587e+08  
2              54.900154      16.469125        1.688893  3.528558e+08  
3              53.230618      16.510225        1.712217  3.559406e+08  
4              55.452032      15.968165        1.691048  3.546039e+08  


C:\Users\muhammad.sannan\AppData\Local\Temp\ipykernel_21536\4263218222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3['Onnet_MOUs'] = dataset3['Onnet_MOUs']*(1/30)
C:\Users\muhammad.sannan\AppData\Local\Temp\ipykernel_21536\4263218222.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3['Offnet_MOUs'] = dataset3['Offnet_MOUs']*(1/30)
